In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp importers.whatsapp

In [ ]:
# export
from hashlib import sha256
from integrators.data.schema import *
from integrators.imports import *
from integrators.importers.importer import ImporterBase
from integrators.importers.matrix import *
from integrators.pod.client import PodClient
from nbdev.showdoc import show_doc
import docker
import string
import subprocess
import sys
import time

In [ ]:
# export

# TODO: remove after it is included in the util 
def get_g_attr(item, name, data_type, default_value=None):
    # hide
    first_or_default = next((att for att in item.genericAttribute if att.name == name), None)
    if first_or_default == None:
        return default_value
    else:
        if data_type == 'int':
            return first_or_default.intValue
        elif data_type == 'bool':
            return first_or_default.boolValue
        elif data_type == 'float':
            return first_or_default.floatValue
        elif data_type == 'string':
            return first_or_default.stringValue
        elif data_type == 'datetime':
            return first_or_default.stringValue
        else:
            raise Exception(f"datatype {data_type} is not supported")
             
def get_random_alphanumeric_string(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    return result_str

# WhatsApp Importer

This importer fetches your WhatsApp data such as contacts, chats and messages (including media files) from the WhatsApp server via a toolset of Matrix homeserver + service bridges. It contains two major components: (1) MatrixClient for communication with the Matrix homeserver via its Client-Server API, and (2) WhatsAppImporter for handling the logic of fetching data from WhatsApp server and creating data items/edges to be uploaded to the Pod. 

Before the importer can start fetching data, the toolset of [Matrix homeserver Synapse](https://github.com/matrix-org/synapse) + [WhatsApp bridge](https://github.com/tulir/mautrix-whatsapp) need to be set up in advance. A Matrix user should be created, the username and token of the created user will be used to authenticate with Matrix homeserver. Additionally, a third-party web client for Matrix is needed (e.g., [Riot/Element](https://app.element.io/)) for now, in order to use the bridge and authenticate with WhatsApp server. 

**Note that the authentication with WhatsApp server relies on WhatsApp web feature, meaning that you need to scan the QR code with the WhatsApp mobile app.** 

**To invite the bridge bot to a Matrix room and authenticate with WhatsApp web:**
* Open in a web browser the Matrix client [Element](https://app.element.io/#/login), change the homeserver URL to your own URL used for homeserver installation (e.g. *http://localhost:8008* for a local installation).
* Use the username and password for registration to login.
* Create a new private group chat, remember to **disable** end-to-end encryption for this room (as it does not work yet for bridge bots).
* Invite bridge bot to the room by sending the message<br/> `/invite @whatsappbot:$your_homeserver_domain`
* Send a message to login `login`
* Use the WhatsApp web on your phone app to scan the shown QR code for authentication.
* **Accept all invitations** to the rooms once logged in.

## MautrixWhatsapp

This class is used for setting up the Mautrix-WhatsApp bridge, including generating configuration files and launching the bridge.

In [ ]:
# export
class MautrixWhatsapp:
    
    def __init__(self, hostname, bridge, user, data_dir, client, my_uid, my_gid):
        self.hostname = hostname # Matrix homeserver hostname
        self.bridge = bridge # bridge domain
        self.user = user # matrix username
        self.dir = data_dir # directory for configuration files
        self.client = client # docker client
        self.my_uid = my_uid # non-root user uid
        self.my_gid = my_gid # non-root user gid
        
        assert self.hostname is not None
        assert self.bridge is not None
        assert self.user is not None
        assert self.dir is not None
        assert self.client is not None
        assert self.my_uid is not None
        assert self.my_gid is not None

    def config_whatsapp_bridge(self):      
        """Generate config files for whatsapp bridge"""
        AS_TOKEN = get_random_alphanumeric_string(64)
        HS_TOKEN = get_random_alphanumeric_string(64)
        SENDER_L = get_random_alphanumeric_string(32)

        find_replace = {
            "localhost": f"{self.bridge}",
            "example.com": f"{self.hostname}",
            "as_token:": f"as_token: {AS_TOKEN}",
            "hs_token:": f"hs_token: {HS_TOKEN}",
        }
        with open('../examples/whatsapp_bridge/example-config.yaml') as fin:
            with open(f"{self.dir}/config.yaml", 'w') as fout:
                for line in fin:
                    for key in find_replace:
                        if key in line:
                            line = line.replace(key, find_replace[key])
                    fout.write(line)
        find_replace = {
            "localhost": f"{self.hostname}",
            "example.com": f"{self.bridge}",
            "as_token:": f"as_token: {AS_TOKEN}",
            "hs_token:": f"hs_token: {HS_TOKEN}",
            "sender_localpart:": f"sender_localpart: {SENDER_L}"
        }
        with open('../examples/whatsapp_bridge/example-registration.yaml') as fin:
            with open(f"{self.dir}/registration.yaml", 'w') as fout:
                for line in fin:
                    for key in find_replace:
                        if key in line:
                            line = line.replace(key, find_replace[key])
                    fout.write(line)
                    
    def run_bridge(self, networkname):
        """Launch whatsapp bridge"""
        self.client.containers.run(
            "dock.mau.dev/tulir/mautrix-whatsapp:latest", 
            detach=True, 
            network=networkname,
            restart_policy={'Name': 'on-failure'},
            ports={'29318': '29318'},
            volumes={self.dir: {'bind': '/data', 'mode': 'rw'}},
            environment=[f"UID={self.my_uid}", f"GID={self.my_gid}"],
            name=self.bridge
        )      
        time.sleep(3)

In [ ]:
show_doc(MautrixWhatsapp.config_whatsapp_bridge)

<h4 id="MautrixWhatsapp.config_whatsapp_bridge" class="doc_header"><code>MautrixWhatsapp.config_whatsapp_bridge</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>MautrixWhatsapp.config_whatsapp_bridge</code>()

Generate config files for whatsapp bridge

In [ ]:
show_doc(MautrixWhatsapp.run_bridge)

<h4 id="MautrixWhatsapp.run_bridge" class="doc_header"><code>MautrixWhatsapp.run_bridge</code><a href="__main__.py#L55" class="source_link" style="float:right">[source]</a></h4>

> <code>MautrixWhatsapp.run_bridge</code>(**`networkname`**)

Launch whatsapp bridge

## WhatsAppImporter

This class provides methods to the frontend for calling the methods in the MatrixClient class. It also handles the processing of the fetched data, and creates data items to be uploaded to the Pod.

WhatsAppImporter takes the following parameters:

* **matrix**: Matrix instance
* **matrix_client**: MatrixClient instance
* **hostname**: Hostname of Matrix homeserver 
* **matrix_address**: URL of Matrix homeserver
* **prefix_service**: Prefix of Matrix username, indicating the service
* **bot_name**: Name of whatsappbot
* **username**: Registered Matrix username
* **password**: Password of registered Matrix user
* **dir**: Directory to store configuration and database files
* **matrix_acc**: Matrix account for registered user
* **matrix_token**: Token associated with Matrix user

In [ ]:
# export
class WhatsAppImporter(ImporterBase):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.matrix = None 
        self.matrix_client = None 
        self.hostname = None 
        self.matrix_address = None 
        self.prefix_service = None 
        self.bot_name = None 
        self.username = None 
        self.password = None 
        self.dir = None 
        self.matrix_acc = None 
        self.matrix_token = None 
        self.acc_idx = {} # helper dict for contacts
        self.msgchan_idx = {} # helper dict for chats
        self.msg_idx = {} # helper dict for messages
        
    def set_matrix_client(self, pod_client, importer_run):
        """Set Matrix client instance and other parameters from importer_run"""
        self.hostname = get_g_attr(importer_run, 'host', 'string', None)
        self.bridgename = get_g_attr(importer_run, 'bridge', 'string', None)
        self.matrix_address = get_g_attr(importer_run, 'address', 'string', None)
        self.prefix_service = get_g_attr(importer_run, 'prefix', 'string', None)
        self.bot_name = get_g_attr(importer_run, 'bot', 'string', None)
        self.username = importer_run.username
        self.password = importer_run.password
        self.dir = f"{os.getcwd()}/../{self.username}_matrix_data"
        
        assert self.hostname is not None 
        assert self.matrix_address is not None 
        assert self.prefix_service is not None 
        assert self.bot_name is not None
        
        # create docker network bridge
        client = docker.from_env()
        network_name = 'matrix-net'
        if not client.networks.list(network_name):
            client.networks.create(network_name, driver='bridge') 
        
        # configure WhatsApp bridge        
        if not os.path.exists(self.dir):
            os.makedirs(self.dir)
        self.mautrix = MautrixWhatsapp(self.hostname, self.bridgename, self.username, self.dir, client, os.getuid(), os.getgid())
        self.mautrix.config_whatsapp_bridge()
        
        # check if matrix is already running
        if not client.containers.list(filters={'name': self.hostname}): # matrix is NOT running
            matrix_item = pod_client.search_by_fields({"externalId": self.hostname})
            self.matrix = Matrix(self.matrix_address, self.hostname, self.username, self.dir, client, os.getuid(), os.getgid())
            if len(matrix_item) == 0:
                self.matrix.config_matrix()
                self.matrix.upload_configs(pod_client)
            else:
                # TODO: find a way to download files
                print("Need to download config files")

            # run matrix and register
            self.matrix.run_matrix(network_name)
            self.matrix.register_user(self.password, pod_client)
            
        # retreave matrix account and token
        items = pod_client.search_by_fields({"externalId": self.username})
        account_item = items[0]
        self.matrix_acc = account_item.displayName
        self.matrix_token = account_item.importJson
        
        # run whatsapp bridge
        self.mautrix.run_bridge(network_name)
            
        # create MatrixClient
        self.matrix_client = MatrixClient(self.matrix_address, self.matrix_token)
    
    def get_receivers(self, room):
        """Fetch message receivers of a room"""
        joined_members = self.matrix_client.get_joined_members(room)
        joined_members = list(joined_members.keys())
        if self.username in joined_members: # except the Matrix user
            joined_members.remove(self.username) 
        # get associated Account item for each user
        receivers = [self.acc_idx[m] for m in joined_members] 
        return receivers
    
    def get_bot_room_id(self, joined_rooms):
        """Get room_id of whatsappbot joined room"""
        for room in joined_rooms:
            joined_members = self.matrix_client.get_joined_members(room)
            # the room shared with whatsappbot has 2 members 
            if len(joined_members) == 2 and self.bot_name in joined_members:
                return room
        
    def bot_list_contacts(self, room_id):
        """Create and send a messge, asking whatsappbot to list all contacts"""
        body = {"msgtype":"m.text", "body":"list contacts"}
        event_id = self.matrix_client.send_messages(room_id, body)
        return event_id

    def get_contacts(self, all_rooms):
        """Fetch a list of contacts from whatsappbot's response"""
        room_id = self.get_bot_room_id(all_rooms)
        event_id = self.bot_list_contacts(room_id)
        time.sleep(2) # wait for whatsappbot's reply
        contact_list = self.matrix_client.get_event_context(room_id, event_id)
        contacts = contact_list[0]["content"]["body"].split("\n")        
        numbers = [self.get_phone_number(c) for c in contacts]
        numbers = [x for x in numbers if x is not None]
        users = [f"{self.prefix_service}{n}:{self.hostname}" for n in numbers]
        return users
    
    @staticmethod
    def get_phone_number(contact):
        """Get phone number from contact information"""
        if not contact.startswith(("#", "* /")):
            parts = contact.split(' - ')
            if len(parts) >= 2:
                phone_number = parts[1][1:-1]
                return phone_number       
            
    def create_account(self, user_id):
        """Create Account item for each user_id"""
        profile = self.matrix_client.get_profile(user_id)
        avatar_url = None
        if "avatar_url" in profile:
            avatar_url = profile["avatar_url"]
        account = Account(externalId=user_id, displayName=profile["displayname"], avatarUrl=avatar_url, service="whatsapp")
        return account

    def create_message_channel(self, room_id, member_accounts):
        """Create MessageChannel item for each room, link with Account items"""
        room_state = self.matrix_client.get_room_state(room_id)
        room_name = None
        room_topic = None
        for s in room_state:
            if s["type"] == "m.room.name":
                room_name = s["content"]["name"]
            if s["type"] == "m.room.topic":
                room_topic == s["content"]["topic"]
        message_channel = MessageChannel(externalId=room_id, name=room_name, topic=room_topic)
        for m in member_accounts:
            message_channel.add_edge("receiver", m) # link with Account
        return message_channel
    
    def create_message(self, event, room):         
        """Create Message item for each event, link with MessageChannel and Account"""
        message = Message(externalId=event["event_id"], importJson=json.dumps(event["content"]), service="whatsapp")
        if not event["sender"] in self.acc_idx:
            account = self.create_account(event["sender"])
            self.acc_idx[event["sender"]] = account
            pod_client.create(account)
        message.add_edge("messageChannel", self.msgchan_idx[room]) # link with MessageChannel
        message.add_edge("sender", self.acc_idx[event["sender"]]) # link with Account
        
        # Link with Message to create a thread
#         if "m.relates_to" in event["content"]:
#             message.add_edge("replyTo", self.msg_idx[event["content"]["m.relates_to"]["m.in_reply_to"]["event_id"]]) 

        # Create media item and link with Message
#         if "info" in event["content"]:
#             media = self.create_media(event["content"])
#             pod_client.create(media)

#             if event["content"]["msgtype"] == "m.video":
#                 message.add_edge("video", media)
#             elif event["content"]["msgtype"] == "m.image":
#                 message.add_edge("photo", media)
#             elif event["content"]["msgtype"] == "m.audio":
#                 message.add_edge("audio", media)
#             elif event["content"]["msgtype"] == "m.file":
#                 message.add_edge("document", media)
        return message
    
    def create_media(self, content):
        """Create media item in different types, link with File"""
        uri = content["url"].split('/')[3]
        binaries = self.matrix_client.download_file(uri)
        sha_file = sha256(binaries).hexdigest()
        # Create File item
        file = File(externalId=content["body"], sha256=sha_file)
        pod_client.create(file)
        pod_client.upload_file(binaries)

        if content["msgtype"] == "m.image":
            photo = Photo(externalId=content["url"])
            photo.add_edge("file", file)
            return photo
        elif content["msgtype"] == "m.video":
            video = Video(externalId=content["url"], duration=content["info"]["duration"])
            video.add_edge("file", file)
            return video
        elif content["msgtype"] == "m.audio":
            audio = Audio(externalId=content["url"], duration=content["info"]["duration"])
            audio.add_edge("file", file)
            return audio
        elif content["msgtype"] == "m.file":
            document = Document(externalId=content["url"], size=content["info"]["size"])
            document.add_edge("file", file)
            return document
        
    def import_all_accounts(self, all_rooms, users):
        """Import all created Account items to Pod"""       
        for r in all_rooms:
            joined_members = self.matrix_client.get_joined_members(r)
            for m in joined_members:
                if not m in users:
                    users.append(m) # add whatsappbot, user 
        for n in users:
            if not n in self.acc_idx:
                account = self.create_account(n)
                self.acc_idx[n] = account
                pod_client.create(account) # upload to Pod   
    
    def import_all_messagechannels(self, all_rooms):
        """Import all created MessageChannel items to Pod"""
        for r in all_rooms:
            if not r in self.msgchan_idx:
                member_accounts = self.get_receivers(r)
                message_channel = self.create_message_channel(r, member_accounts)
                self.msgchan_idx[r] = message_channel 
                pod_client.create(message_channel) # upload to Pod       
                pod_client.create_edges(message_channel.get_all_edges())
    
    def import_all_messages(self, next_batch):
        """Import all created Message items to Pod"""
        sync_events = self.matrix_client.sync_events(next_batch)
        # messages from joined rooms 
        joined_rooms = sync_events["rooms"]["join"]
        for r in joined_rooms:
            room_events = sync_events["rooms"]["join"][r]["timeline"]["events"]
            for e in room_events:
                if not e["event_id"] in self.msg_idx:
                    message = self.create_message(e, r)
                    self.msg_idx[e["event_id"]] = message
                    pod_client.create(message) # upload to Pod
                    pod_client.create_edges(message.get_all_edges())
        return sync_events["next_batch"]
                
    def run(self, importer_run, pod_client=None, verbose=True):
        """This is the main function of WhatsAppImporter, which runs based on the information of importer_run."""
        self.set_matrix_client(pod_client, importer_run)
        self.update_run_status(pod_client, importer_run, "running")
        
        all_rooms = self.matrix_client.get_joined_rooms()
        notified = False
        while not len(all_rooms) > 1: # wait for web authentication 
            if not notified:
                self.update_run_status(pod_client, importer_run, "waiting for web authentication")
                print(f"Please login to {self.matrix_address} with username {self.username} and password {self.password}, and invite {self.bot_name} to a new room.")
                notified = True
            time.sleep(10)
            all_rooms = self.matrix_client.get_joined_rooms()
            
        users = self.get_contacts(all_rooms)
        next_batch = "s9_7_0_1_1_1"
        
        while True: # polling for new contacts, chats and messages    
            all_rooms = self.matrix_client.get_joined_rooms()       
            
            self.update_progress_message(pod_client, importer_run, "importing contacts", verbose=verbose)
            self.import_all_accounts(all_rooms, users) 
            
            self.update_progress_message(pod_client, importer_run, "importing chats", verbose=verbose)
            self.import_all_messagechannels(all_rooms)

            self.update_progress_message(pod_client, importer_run, "importing messages", verbose=verbose)
            next_batch = self.import_all_messages(next_batch)

            self.update_run_status(pod_client, importer_run, "polling")
            time.sleep(2)

In [ ]:
show_doc(WhatsAppImporter.create_account)

<h4 id="WhatsAppImporter.create_account" class="doc_header"><code>WhatsAppImporter.create_account</code><a href="__main__.py#L70" class="source_link" style="float:right">[source]</a></h4>

> <code>WhatsAppImporter.create_account</code>(**`user_id`**)

Create Account item for each user_id

In [ ]:
show_doc(WhatsAppImporter.create_message_channel)

<h4 id="WhatsAppImporter.create_message_channel" class="doc_header"><code>WhatsAppImporter.create_message_channel</code><a href="__main__.py#L79" class="source_link" style="float:right">[source]</a></h4>

> <code>WhatsAppImporter.create_message_channel</code>(**`room_id`**, **`member_accounts`**)

Create MessageChannel item for each room, link with Account items

In [ ]:
show_doc(WhatsAppImporter.create_message)

<h4 id="WhatsAppImporter.create_message" class="doc_header"><code>WhatsAppImporter.create_message</code><a href="__main__.py#L94" class="source_link" style="float:right">[source]</a></h4>

> <code>WhatsAppImporter.create_message</code>(**`event`**, **`room`**)

Create Message item for each event, link with MessageChannel and Account

In [ ]:
show_doc(WhatsAppImporter.create_media)

<h4 id="WhatsAppImporter.create_media" class="doc_header"><code>WhatsAppImporter.create_media</code><a href="__main__.py#L120" class="source_link" style="float:right">[source]</a></h4>

> <code>WhatsAppImporter.create_media</code>(**`content`**)

Create media item in different types, link with File

In [ ]:
show_doc(WhatsAppImporter.import_all_accounts)

<h4 id="WhatsAppImporter.get_all_accounts" class="doc_header"><code>WhatsAppImporter.get_all_accounts</code><a href="__main__.py#L147" class="source_link" style="float:right">[source]</a></h4>

> <code>WhatsAppImporter.get_all_accounts</code>(**`all_rooms`**)

Fetch all created Account items

In [ ]:
show_doc(WhatsAppImporter.import_all_messagechannels)

<h4 id="WhatsAppImporter.get_all_messagechannels" class="doc_header"><code>WhatsAppImporter.get_all_messagechannels</code><a href="__main__.py#L169" class="source_link" style="float:right">[source]</a></h4>

> <code>WhatsAppImporter.get_all_messagechannels</code>(**`all_rooms`**)

Fetch all created MessageChannel items, upload to Pod

In [ ]:
show_doc(WhatsAppImporter.import_all_messages)

<h4 id="WhatsAppImporter.get_all_messages" class="doc_header"><code>WhatsAppImporter.get_all_messages</code><a href="__main__.py#L183" class="source_link" style="float:right">[source]</a></h4>

> <code>WhatsAppImporter.get_all_messages</code>()

Fetch all created Message items, upload to Pod

In [ ]:
show_doc(WhatsAppImporter.run)

<h4 id="WhatsAppImporter.run" class="doc_header"><code>WhatsAppImporter.run</code><a href="__main__.py#L198" class="source_link" style="float:right">[source]</a></h4>

> <code>WhatsAppImporter.run</code>(**`importer_run`**, **`pod_client`**=*`None`*, **`verbose`**=*`True`*)

This is the main function of WhatsAppImporter, which runs based on the information of importer_run.

# Tests

## Without Matrix homeserver+bridge

Create MessageChannel and Message items, add edge in between, upload to Pod.

In [ ]:
# slow
DB_KEY = "2DD29CA851E7B56E4697B0E1F08507293D761A05CE4D1B628663F411A8086D99"
OWNER_KEY = "2DD29CA851E7B56E4697B0E1F08507293D761A05CE4D1B628663F411A8086D99"

pod_client = PodClient(database_key=DB_KEY, owner_key=OWNER_KEY)
pod_client.delete_all()

message_channel = MessageChannel(externalId="!scbkDPfWMZqpqyjnsS:synapse", name="Test", topic="Private chat")
pod_client.create(message_channel)

event = {'type': 'm.room.message', 'sender': '@foo:synapse', 'content': {'msgtype': 'm.text', 'body': 'list contacts'}, 'origin_server_ts': 1606235079767, 'unsigned': {'age': 59529472}, 'event_id': '$biXu9En8lvI7gxfV1O6_4SxgBtI6HpcTMr6oZtEJSz4'}
message = Message(externalId=event["event_id"], importJson=json.dumps(event["content"]), service="whatsapp")
message.add_edge("messageChannel", message_channel)
pod_client.create(message)
pod_client.create_edges(message.get_all_edges())

assert pod_client.external_id_exists(message_channel) == True
assert pod_client.external_id_exists(message) == True

pod_client.delete_all()

## With Matrix homeserver+bridge

**Require to first register with Matrix, update username and token.**

Import contacts, chats and messages and upload to Pod.

In [ ]:
# slow
DB_KEY = "2DD29CA851E7B56E4697B0E1F08507293D761A05CE4D1B628663F411A8086D99"
OWNER_KEY = "2DD29CA851E7B56E4697B0E1F08507293D761A05CE4D1B628663F411A8086D99"
USERNAME = OWNER_KEY[0:10]
PASSWORD = get_random_alphanumeric_string(10)
HOSTNAME = f"{USERNAME}-synapse"
BRIDGENAME = f"{USERNAME}-mautrix-whatsapp"
DEFAULT_MATRIX_ADDRESS = "http://localhost:8008"
PREFIX_SERVICE = "@whatsapp_"
BOT_NAME = f"@whatsappbot:{HOSTNAME}"


pod_client = PodClient(database_key=DB_KEY, owner_key=OWNER_KEY)

whatsapp_importer = WhatsAppImporter(name="whatsapp")
pod_client.create(whatsapp_importer)

ga_host = GenericAttribute(name='host', stringValue=HOSTNAME)
ga_bridge = GenericAttribute(name='bridge', stringValue=BRIDGENAME)
ga_address = GenericAttribute(name='address', stringValue=DEFAULT_MATRIX_ADDRESS)
ga_prefix = GenericAttribute(name='prefix', stringValue=PREFIX_SERVICE)
ga_bot = GenericAttribute(name='bot', stringValue=BOT_NAME)
pod_client.create(ga_host)
pod_client.create(ga_bridge)
pod_client.create(ga_address)
pod_client.create(ga_prefix)
pod_client.create(ga_bot)

importer_run = ImporterRun(progress=0, username=USERNAME, password=PASSWORD, repository="memri-pyintegrator")
importer_run.add_edge('genericAttribute', ga_host)
importer_run.add_edge('genericAttribute', ga_bridge)
importer_run.add_edge('genericAttribute', ga_address)
importer_run.add_edge('genericAttribute', ga_prefix)
importer_run.add_edge('genericAttribute', ga_bot)
importer_run.add_edge('importer', whatsapp_importer)
pod_client.create(importer_run)
pod_client.create_edges(importer_run.get_all_edges())

whatsapp_importer.run(importer_run=importer_run, pod_client=pod_client)

## Call run_importer interface of the Pod


In [ ]:
# slow
DB_KEY = "2DD29CA851E7B56E4697B0E1F08507293D761A05CE4D1B628663F411A8086D99"
OWNER_KEY = "2DD29CA851E7B56E4697B0E1F08507293D761A05CE4D1B628663F411A8086D99"

pod_client = PodClient(database_key=DB_KEY, owner_key=OWNER_KEY)


query = {
            'databaseKey':pod_client.database_key,
            'payload':{
                 'uid':importer_run.uid,
                 'servicePayload': {
                     'databaseKey':pod_client.database_key,
                     'ownerKey':pod_client.owner_key
                  },
            }
       }

print(importer_run.uid)
print(requests.post(f'http://0.0.0.0:3030/v2/{pod_client.owner_key}/run_importer',
                   json=query).content)

In [ ]:
# hide
# TODO: test for media upload

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted importers.EmailImporter.ipynb.
Converted importers.Importer.ipynb.
Converted importers.MatrixClient.ipynb.
Converted importers.WhatsAppImporter.ipynb.
Converted importers.util.ipynb.
Converted index.ipynb.
Converted indexers.FaceRecognitionIndexer.ipynb.
Converted indexers.FacerecognitionIndexer.Photo.ipynb.
Converted indexers.GeoIndexer.ipynb.
Converted indexers.NoteListIndexer.NoteList.ipynb.
Converted indexers.NoteListIndexer.Parser.ipynb.
Converted indexers.NoteListIndexer.ipynb.
Converted indexers.NoteListIndexer.util.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted pod.client.ipynb.
